In [1]:
from datetime import datetime
import dask.dataframe as dd
import os
import pathlib as pl
import sys



from dask.distributed import Client
import nltk.data
from pyprojroot import here


In [2]:
client = Client(n_workers=6, threads_per_worker=1, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:61945 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 6.00 GB


In [3]:
nparts = 1000
fout = str(here(f"./data/db/working/kaggle/id_model_inputs/01-01-dask_data_frame_partitions_{nparts}.json.gzip", warn=False))
fout

'C:\\Users\\Danie\\git\\hub\\databridge\\covid19\\data\\db\\working\\kaggle\\id_model_inputs\\01-01-dask_data_frame_partitions_1000.json.gzip'

In [4]:
pth = os.path.join(fout, "*.part")
ddf = dd.read_json(pth, compression="gzip")

In [5]:
ddf

,pid,num_authors,title,text
npartitions=985,,,,
,object,int64,object,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
# break up into sentences
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle') # nltk.download('punkt')

In [7]:
ddf["text_sent_lower"] = ddf["text"].map_partitions(
    lambda x: x.apply(
        lambda y: sent_detector.tokenize(y.lower().strip())
    )
)

In [8]:
#ddf.visualize()

In [9]:
start_time = datetime.now()

pl.Path(here("./data/db/working/kaggle/id_model_inputs", warn=False)).mkdir(parents=True, exist_ok=True)
ddf.to_json(str(here("./data/db/working/kaggle/id_model_inputs/01-02-split_sentences.json.gzip", warn=False)), compression="gzip") # about 5 minutes

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:05:02.213197


In [10]:
client.shutdown()